In [89]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from sklearn.metrics import accuracy_score
from itertools import product
from random import random, uniform
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from keras.callbacks import Callback
import pandas as pd
import math
import copy
data=pd.read_csv(r"C:\Users\Gabriel\OneDrive\Documents\loan_preprocessed.csv")
 
X = data.drop('Personal Loan',axis=1).values
Y = data['Personal Loan'].values
X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.33, random_state=42)


def create_model():
    model = Sequential()
    model.add(Dense(4, input_dim=12, activation='sigmoid'))
    model.add(Dense(1, activation='sigmoid'))
    return model


In [90]:
X_train.shape,y_train.shape,X_test.shape,y_test.shape

((3350, 12), (3350,), (1650, 12), (1650,))

In [91]:
model = create_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 4)                 52        
                                                                 
 dense_15 (Dense)            (None, 1)                 5         
                                                                 
Total params: 57
Trainable params: 57
Non-trainable params: 0
_________________________________________________________________


In [76]:

def fitness_function(model, weights):
    model.set_weights(weights)
    inputs = X
    targets = Y.reshape(-1,1)
    model.compile(loss='binary_crossentropy', optimizer=SGD(lr=0.1), metrics=['accuracy'])
    class FitnessCallback(Callback):
        def on_epoch_end(self, epoch, logs={}):
            self.fitness_score = logs.get('accuracy')

    fitness_callback = FitnessCallback()
    model.fit(inputs, targets, epochs=1, batch_size=1, callbacks=[fitness_callback], verbose=0)
    y_pred = np.round(model.predict(inputs).ravel())
    accuracy = accuracy_score(targets, y_pred)
    return accuracy



In [77]:

def genetic_algorithm(model, pop_size=50, num_generations=100, mutation_rate=0.1):
    # Initialize the population with random weights
    population = [model.get_weights() for i in range(pop_size)]

    for gen in range(num_generations):
        # Evaluate the fitness of each candidate solution
        fitness = [fitness_function(model, weights) for weights in population]

        # Print the best fitness in the current generation
        best_fitness = max(fitness)
        print(f"Generation {gen}: Best Fitness = {best_fitness}")

        # Select the parents for crossover using roulette wheel selection
        fitness_sum = sum(fitness)
        parents = []
        for i in range(pop_size):
            cumulative_fitness = 0
            for j in range(pop_size):
                cumulative_fitness += fitness[j]
                if random() < (cumulative_fitness / fitness_sum):
                    parents.append(population[j])
                    break

        # Generate the offspring through single-point crossover
        offspring = []
        parent1=[]
        parent2=[]
        for i in range(0, pop_size, 2):
            parent1 = parents[i]
            parent2 = parents[i+1]
            crossover_point = np.random.randint(0, len(parent1))
            print(parent1,parent2)
            p1=parent1[:crossover_point]
            p2=parent2[crossover_point:]
            p3=parent2[:crossover_point]
            p4=parent1[crossover_point:]
            child1=p1+p2
            child2=p3+p4
            
#             child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
#             child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            offspring.append(child1)
            offspring.append(child2)

        # Mutate the offspring by randomly perturbing each weight
        for i in range(pop_size):
            for j in range(len(offspring[i])):
                if random() < mutation_rate:
                    offspring[i][j] += uniform(-0.5, 0.5)

        # Combine the parents and offspring
        combined_population = parents + offspring

        # Select the next generation using elitist truncation selection
        combined_fitness = [fitness_function(model, weights) for weights in combined_population]
        sorted_indices = np.argsort(combined_fitness)[::-1]
        population = [combined_population[i] for i in sorted_indices[:pop_size]]

        # Return the best weights found by the genetic algorithm
        best_weights = population[np.argmax([fitness_function(model, weights) for weights in population])]
        return best_weights
        


In [78]:
print("Training with genetic algorithm...")
ga_weights = genetic_algorithm(model, pop_size=50, num_generations=100)
model.set_weights(ga_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))

Training with genetic algorithm...
157/157 [==============================] - 0s 1ms/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step


...

157/157 [==============================] - 0s 873us/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 740us/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


52/52 [==============================] - 0s 926us/step
Accuracy on test set: 0.8872727272727273


In [26]:
def cultural_algorithm(model, pop_size=50, num_generations=100, mutation_rate=0.1, num_cultures=10, culture_range=0.1):
# Initialize the cultures with random weights
    population=[]
    population = [model.get_weights() for i in range(pop_size)]
    cultures = [model.get_weights() for i in range(num_cultures)]
    for gen in range(num_generations):
        # Evaluate the fitness of each candidate solution in each culture
        fitness = [[] for i in range(num_cultures)]
        for i in range(pop_size):
#             print(population)
            weights = population[i]
            print(weights)
            culture_index = np.argmin([np.linalg.norm(weights - culture_weights) for culture_weights in cultures])
            fitness[culture_index].append(fitness_function(model, weights))
    
        # Print the best fitness in each culture in the current generation
        for i in range(num_cultures):
            best_fitness = max(fitness[i])
            print(f"Generation {gen}: Culture {i}: Best Fitness = {best_fitness}")
    
        # Update the cultures by selecting the best candidate solution in each culture
        for i in range(num_cultures):
            culture_fitness = fitness[i]
            best_index = np.argmax(culture_fitness)
            best_weights = population[best_index]
            cultures[i] = best_weights
    
        # Select the parents for crossover using roulette wheel selection
        fitness_sum = sum([len(fitness[i]) for i in range(num_cultures)])
        parents = []
        for i in range(pop_size):
            cumulative_fitness = 0
            for j in range(num_cultures):
                cumulative_fitness += len(fitness[j])
                if random() < (cumulative_fitness / fitness_sum):
                    culture_weights = cultures[j]
                    candidate_index = np.random.choice(len(fitness[j]))
                    candidate_weights = population[candidate_index]
                    parents.append((candidate_weights, culture_weights))
                    break
    
        # Generate the offspring through single-point crossover
        offspring = []
        for i in range(0, pop_size, 2):
            parent1, culture1 = parents[i]
            parent2, culture2 = parents[i+1]
            crossover_point = np.random.randint(0, len(parent1))
            child1 = np.concatenate((parent1[:crossover_point], parent2[crossover_point:]))
            child2 = np.concatenate((parent2[:crossover_point], parent1[crossover_point:]))
            offspring.append(child1)
            offspring.append(child2)
    
        # Mutate the offspring by randomly perturbing each weight
        for i in range(pop_size):
            for j in range(len(offspring[i])):
                if random() < mutation_rate:
                    offspring[i][j] += uniform(-culture_range, culture_range)
    
        # Combine the parents and offspring
        combined_population = parents + offspring
    
        # Select the next generation using elitist truncation selection
        combined_fitness = [fitness_function(model, weights) for weights in combined_population]
        sorted_indices = np.argsort(combined_fitness)[::-1]
        population = [combined_population[i] for i in sorted_indices[:pop_size]]
    
    # Return the best weights found by the cultural algorithm
    best_weights = population[np.argmax([fitness_function(model,weights) for weights in population])]
    return best_weights



In [27]:
print("Training with cultural algorithm...")
cultural_weights = cultural_algorithm(model, pop_size=50, num_generations=100)
model.set_weights(cultural_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))

Training with cultural algorithm...
[array([[ 0.08079499, -0.17892185,  0.24507862,  0.44195843,  0.25272733,
        -0.3902816 ,  0.3129056 ,  0.00383943, -0.42533424, -0.01766419],
       [ 0.31299204,  0.3723052 , -0.13920173, -0.4314954 ,  0.3888772 ,
         0.15908253,  0.4265927 ,  0.3585871 , -0.08176398, -0.29027858],
       [-0.33459687, -0.00365859,  0.03485316, -0.3769537 ,  0.34256124,
        -0.25831634,  0.12813455, -0.3957146 , -0.22756223, -0.0232029 ],
       [ 0.07886583, -0.43907574,  0.41251856, -0.04720321, -0.3805081 ,
         0.16496742, -0.4309401 , -0.14021817,  0.11678988,  0.24636418],
       [-0.04613483,  0.41065305, -0.07113764, -0.36095828,  0.2435475 ,
        -0.06932011, -0.32850406, -0.11865848, -0.24952112, -0.16288367],
       [ 0.21725422, -0.33795893,  0.08155227, -0.08832052, -0.37042627,
        -0.02079052,  0.12709361, -0.02630511, -0.18701598,  0.32419097],
       [ 0.1354571 ,  0.26101613, -0.20333639, -0.24596362,  0.25938332,
        

TypeError: unsupported operand type(s) for -: 'list' and 'list'

In [123]:
def ant_colony_optimization(model, pop_size=50, num_ants=50, num_generations=100, alpha=1.0, beta=3.0, evaporation_rate=0.1):
# Initialize the pheromone matrix with equal values
    num_weights = sum([len(layer.get_weights()[0].flatten()) for layer in model.layers])
    pheromone = np.ones(num_weights)
    
    for gen in range(num_generations):
        # Initialize the ant solutions and their fitness values
        solutions = []
        fitness = []
        for ant in range(num_ants):
            # Generate a candidate solution by applying a random perturbation to the pheromone levels
            perturbation = np.random.normal(scale=1.0, size=num_weights)
            candidate_weights = model.get_weights() + pheromone *perturbation
            solutions.append(candidate_weights)
    
            # Evaluate the fitness of the candidate solution
            fitness.append(fitness_function(model, candidate_weights))
    
        # Print the best fitness in the current generation
        best_fitness = max(fitness)
        print(f"Generation {gen}: Best Fitness = {best_fitness}")
    
        # Update the pheromone matrix by depositing pheromone proportional to the fitness of each solution
        deposit = [fitness[i] / sum(fitness) for i in range(num_ants)]
        pheromone *= (1 - evaporation_rate)
        for i in range(num_ants):
            weight_indices = 0
            for layer in model.layers:
                layer_weights = layer.get_weights()
                for j in range(len(layer_weights)):
                    shape = layer_weights[j].shape
                    num_weights = shape[0] * shape[1]
                    pheromone_indices = range(weight_indices, weight_indices + num_weights)
                    pheromone[pheromone_indices] += alpha * deposit[i] * solutions[i][weight_indices:weight_indices+num_weights].flatten()
                    weight_indices += num_weights
    
    # Return the best weights found by the ant colony optimization algorithm
    best_weights = solutions[np.argmax(fitness)]
    return best_weights

In [124]:
print("Training with ant colony optimization...")
aco_weights = ant_colony_optimization(model, pop_size=50, num_ants=50, num_generations=100)
model.set_weights(aco_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))


Training with ant colony optimization...


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step


...

157/157 [==============================] - 0s 2ms/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


KeyboardInterrupt: 

In [129]:
def particle_swarm_optimization(model, pop_size=50, num_particles=50, num_generations=100, w=0.7, c1=1.494, c2=1.494):
# Initialize the particles with random weights and velocities
    particles = []
    velocities = []
    for i in range(num_particles):
        weights = model.get_weights()
    particles.append(weights)
    particles_best = copy.deepcopy(particles)
    particles_best_fitness = [fitness_function(model, i) for i in particles]
    
    velocities.append(np.zeros_like(weights))
    
    # Initialize the global best position and its fitness value
    global_best_position = particles[0]
    global_best_fitness = fitness_function(model, global_best_position)
    
    for gen in range(num_generations):
        # Update the velocity and position of each particle
        for i in range(num_particles):
            # Update the velocity of the particle
            for k in range(len(particles[0])):
                new_velocity = w * velocities[k]
                print(new_velocity.shape,velocities[k].shape)
                cognitive_velocity = c1 * np.random.rand() * (particles[i][k] - particles_best[i][k])
                social_velocity = c2 * np.random.rand() * (global_best_position[k] - particles[i][k])
                new_velocity += cognitive_velocity + social_velocity
                velocities[k] = new_velocity
    
            # Update the position of the particle
            new_position = particles[i] + new_velocity
            particles[i] = new_position
    
            # Update the best position of the particle
            particle_fitness = fitness_function(model, particles[i])
            if particle_fitness > particles_best_fitness[i]:
                particles_best[i] = particles[i]
                particles_best_fitness[i] = particle_fitness
    
            # Update the global best position
            if particle_fitness > global_best_fitness:
                global_best_position = particles[i]
                global_best_fitness = particle_fitness
    
        # Print the best fitness in the current generation
        print(f"Generation {gen}: Best Fitness = {global_best_fitness}")
    
    # Return the best weights found by the particle swarm optimization algorithm
    best_weights = global_best_position
    return best_weights



In [130]:
print("Training with particle swarm optimization...")
pso_weights = particle_swarm_optimization(model, pop_size=50, num_particles=50, num_generations=10)
model.set_weights(pso_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))
   

Training with particle swarm optimization...


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step


<__array_function__ internals>:180: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


157/157 [==============================] - 0s 1ms/step
(4,) (4,)


ValueError: non-broadcastable output operand with shape (4,) doesn't match the broadcast shape (12,4)

In [84]:
                
model = Sequential([
Dense(input_dim=20, units = 10,activation='relu'),
Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

print("Training with genetic algorithm...")
ga_weights = genetic_algorithm(model, pop_size=50, num_generations=100)
model.set_weights(ga_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))

print("Training with cultural algorithm...")
cultural_weights = cultural_algorithm(model, pop_size=50, num_generations=100)
model.set_weights(cultural_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))

print("Training with ant colony optimization...")
aco_weights = ant_colony_optimization(model, pop_size=50, num_ants=50, num_generations=100)
model.set_weights(aco_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))

print("Training with particle swarm optimization...")
pso_weights = particle_swarm_optimization(model, pop_size=50, num_particles=50, num_generations=100)
model.set_weights(pso_weights)
print("Accuracy on test set:", accuracy_score(y_test, model.predict(X_test).round()))
                           


Training with genetic algorithm...


C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\optimizers\legacy\gradient_descent.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


ValueError: in user code:

    File "C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1284, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1268, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1249, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\engine\training.py", line 1050, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\Gabriel\AppData\Roaming\Python\Python39\site-packages\

In [33]:
model

In [66]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_4 (Dense)             (None, 10)                210       
                                                                 
 dense_5 (Dense)             (None, 1)                 11        
                                                                 
Total params: 221
Trainable params: 221
Non-trainable params: 0
_________________________________________________________________


In [35]:
X_test.shape

(1650, 12)